In [ ]:
import pandas as pd
import rdflib

g = rdflib.ConjunctiveGraph()
result = g.parse("mhdb/mhdb.ttl", format='ttl')
MESHg = rdflib.ConjunctiveGraph()
MESH = MESHg.parse("mhdb/MESH.ttl", format='ttl')

In [ ]:
g += MESHg

In [ ]:
for s, p, o in g:
    print(" ".join([s, p, o, "\n"]))

In [ ]:
qres ="""# "What are all of the top-level mental disorders?"
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT ?option ?subclass 
WHERE {
  ?subclass rdfs:subClassOf* ?intermediate .
  ?intermediate rdfs:subClassOf* :MentalDisorder .
  ?subclass rdfs:label ?label .
  OPTIONAL { ?label skos:prefLabel ?pl }
  BIND ( IF ( !bound(?label), ?subclass, IF ( bound(?label), IF ( !bound(?pl), ?label, ?pl), '' ) ) AS ?option )
}
GROUP BY ?option ?subclass
HAVING(count(?intermediate) = 2)
ORDER BY ?option"""
column_names = ['label', 'iri']
sparql_result = g.query(qres)
options = []
for row in sparql_result:
    options.append([row[0], row[1]])
df = pd.DataFrame(options, columns=column_names)
print(df)

for key, value in df.iterrows():
    print([value['label'], value['iri']])

In [ ]:
qres2 = """PREFIX : <http://www.purl.org/mentalhealth#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?subject ?relationship ?object 
FROM <mhdb.ttl>
FROM <http://data.bioontology.org/ontologies/MESH/submissions/13/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb>
WHERE {
       { :Feeding_or_Eating_Disorder ?relationship ?object }
       UNION
       { ?subject ?relationship :Feeding_or_Eating_Disorder }
       BIND ( IF ( bound(?subject), ?subject, :Feeding_or_Eating_Disorder ) AS ?subject )
       BIND ( IF ( bound(?object), ?object, :Feeding_or_Eating_Disorder ) AS ?object )
}
        """
sparql_result = g.query(qres2)
options = []
for row in sparql_result:
    options.append([[row[0]], [row[1]], [row[2]]])
df = pd.DataFrame(options, columns=['subject', 'relationship', 'object'])
print(df)